In [43]:
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from transformers import pipeline, set_seed
from collections import defaultdict
import torch
import transformers
from tqdm.notebook import tqdm

In [7]:
df = pd.read_csv('../data/initial_datasets/cmb_prosocial_labeled.csv')

In [33]:
df = df[~(df['action'] == 'UNCLEAR')]

In [14]:
df.iloc[3717]

match_id                                      24644
player_name                             Shadow Link
player_chat    hope you guys enjoyed your free game
action                                NOT-PROSOCIAL
Name: 3717, dtype: object

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df['player_chat'], df['action'], test_size=0.3)

In [ ]:
X_train

'hope you guys enjoyed your free game'

In [17]:
classifier = transformers.pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:  27%|##7       | 73.4M/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use mps:0


In [35]:
X_train.to_list()

['?. ssh. i appreciate. ur effort. why me?. u guys never learn dont u. k. useless team',
 'gl hf. gg',
 'lost mid. to roski. useless cm at top. lost game. yeyh. cm top > +25',
 'i reatad. wait 3 ins. wait 3 mins. damn. fuck. update. cant fucking conection. nice valve. alright. alright. lol. taht rosh bash. if u keeping this game i will kich all ur ass with dr. nah. waht now putang ina mo bobo nec. so handsome right. so what. not sad ',
 'alchemist 32 minutes. relic. new meta. sure.... alchemist just bought his account. 3k accounts are cheap. only accounts peruvians can afford. 33 minute radiance alchemist. very good farm. really impressed. alchemist is a horrible farmer. 30 min radiance is the norm',
 'did u channel ur ultimate in ur fountain?',
 'stop def :s. gg',
 'gg ez',
 'gg',
 'gg. aw buhat daw. buhat ka gago',
 'ggwp',
 'good support. not u, sf',
 'rs. ez kolt didnot upgrade 1st spell ',
 'rofl no. rekt',
 'finish pls. our cores are shit',
 'hes fucking retarded. not according t

In [ ]:
5000/16

In [36]:
def batch(iterable, batch_size=16):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

results = []
for chunk in tqdm(batch(X_train.to_list(), batch_size=16)):
    chunk_results = classifier(chunk)
    results.extend(chunk_results)


0it [00:00, ?it/s]

In [37]:
results

[{'label': 'POSITIVE', 'score': 0.9771550297737122},
 {'label': 'NEGATIVE', 'score': 0.8782517313957214},
 {'label': 'NEGATIVE', 'score': 0.9997360110282898},
 {'label': 'NEGATIVE', 'score': 0.7307473421096802},
 {'label': 'POSITIVE', 'score': 0.8583253622055054},
 {'label': 'NEGATIVE', 'score': 0.9981558918952942},
 {'label': 'NEGATIVE', 'score': 0.9673581719398499},
 {'label': 'NEGATIVE', 'score': 0.9029054045677185},
 {'label': 'POSITIVE', 'score': 0.5559746026992798},
 {'label': 'NEGATIVE', 'score': 0.9935494065284729},
 {'label': 'NEGATIVE', 'score': 0.9214951395988464},
 {'label': 'POSITIVE', 'score': 0.9807739853858948},
 {'label': 'NEGATIVE', 'score': 0.9884824156761169},
 {'label': 'NEGATIVE', 'score': 0.9853530526161194},
 {'label': 'NEGATIVE', 'score': 0.9995225667953491},
 {'label': 'NEGATIVE', 'score': 0.9977120161056519},
 {'label': 'NEGATIVE', 'score': 0.9961146116256714},
 {'label': 'NEGATIVE', 'score': 0.999713122844696},
 {'label': 'NEGATIVE', 'score': 0.9967556595802

In [40]:
y_act = y_train.replace({"NOT-PROSOCIAL": -1, "PROSOCIAL": 1}).to_list()

/var/folders/lv/pnwq6bmj4tq68bsvy__37qyh0000gn/T/ipykernel_49454/1525439145.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_act = y_train.replace({"NOT-PROSOCIAL": -1, "PROSOCIAL": 1}).to_list()


In [42]:
y_pred = []
for res in results:
    if res['label'] == 'POSITIVE':
        y_pred.append(1)
    else:
        y_pred.append(-1)

In [44]:
accuracy_score(y_pred, y_act)

0.7725768321513002